<a href="https://colab.research.google.com/github/deveshsangwan/Skin-lesions-classification/blob/main/Ensemble_skin_lesions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model, Input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.resnet import ResNet152

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
X_test = np.load('/content/drive/My Drive/Datasets/Skin_npy/x_test.npy')
X_val = np.load('/content/drive/My Drive/Datasets/Skin_npy/x_validate.npy')
y_test = np.load('/content/drive/My Drive/Datasets/Skin_npy/y_test.npy')
y_val = np.load('/content/drive/My Drive/Datasets/Skin_npy/y_validate.npy')

In [6]:
X_val.shape, X_test.shape, y_val.shape, y_test.shape

((802, 192, 256, 3), (2003, 192, 256, 3), (802, 7), (2003, 7))

In [7]:
input_shape = X_val[0,:,:,:].shape
model_input = Input(shape=input_shape)

#Xception

In [8]:
xception = Xception(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

In [9]:
for layer in xception.layers:
    layer.trainable = True

In [10]:
xception_last_layer = xception.get_layer('block14_sepconv2_act')
print('last layer output shape:', xception_last_layer.output_shape)
xception_last_output = xception_last_layer.output

last layer output shape: (None, 6, 8, 2048)


In [11]:
# Flatten the output layer to 1 dimension
x_xception = layers.GlobalMaxPooling2D()(xception_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_xception = layers.Dense(512, activation='relu')(x_xception)
# Add a dropout rate of 0.7
x_xception = layers.Dropout(0.5)(x_xception)
# Add a final sigmoid layer for classification
x_xception = layers.Dense(7, activation='softmax')(x_xception)

# Configure and compile the model

xception_model = Model(model_input, x_xception)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
xception_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [12]:
xception_model.load_weights('/content/drive/My Drive/Xception.h5')

In [13]:
xception_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 95, 127, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 95, 127, 32)  128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 95, 127, 32)  0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

#DenseNet

In [14]:
denseNet = DenseNet201(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

In [15]:
for layer in denseNet.layers:
    layer.trainable = True

In [16]:
denseNet_last_layer = denseNet.get_layer('relu')
print('last layer output shape:', denseNet_last_layer.output_shape)
denseNet_last_output = denseNet_last_layer.output

last layer output shape: (None, 6, 8, 1920)


In [17]:
# Flatten the output layer to 1 dimension
x_denseNet = layers.GlobalMaxPooling2D()(denseNet_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_denseNet = layers.Dense(512, activation='relu')(x_denseNet)
# Add a dropout rate of 0.7
x_denseNet = layers.Dropout(0.5)(x_denseNet)
# Add a final sigmoid layer for classification
x_denseNet = layers.Dense(7, activation='softmax')(x_denseNet)

# Configure and compile the model

denseNet_model = Model(model_input, x_denseNet)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
denseNet_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [18]:
denseNet_model.load_weights("/content/drive/My Drive/DenseNet.h5")

In [19]:
denseNet_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 198, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 96, 128, 64)  9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 96, 128, 64)  256         conv1/conv[0][0]                 
_______________________________________________________________________________________

#InceptionV3

In [20]:
inception = InceptionV3(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

In [21]:
for layer in inception.layers:
    layer.trainable = True

In [22]:
inception_last_layer = inception.get_layer('mixed10')
print('last layer output shape:', inception_last_layer.output_shape)
inception_last_output = inception_last_layer.output

last layer output shape: (None, 4, 6, 2048)


In [23]:
# Flatten the output layer to 1 dimension
x_inception = layers.GlobalMaxPooling2D()(inception_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_inception = layers.Dense(512, activation='relu')(x_inception)
# Add a dropout rate of 0.7
x_inception = layers.Dropout(0.5)(x_inception)
# Add a final sigmoid layer for classification
x_inception = layers.Dense(7, activation='softmax')(x_inception)

# Configure and compile the model

inception_model = Model(model_input, x_inception)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
inception_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [24]:
inception_model.load_weights("/content/drive/My Drive/InceptionV3.h5")

In [25]:
inception_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 95, 127, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 95, 127, 32)  96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 95, 127, 32)  0           batch_normalization_4[0][0]      
_______________________________________________________________________________________

#ResNet152

In [26]:
resnet = ResNet152(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

In [27]:
for layer in resnet.layers:
    layer.trainable = True

In [28]:
resnet_last_layer = resnet.get_layer('conv5_block3_out')
print('last layer output shape:', resnet_last_layer.output_shape)
resnet_last_output = resnet_last_layer.output

last layer output shape: (None, 6, 8, 2048)


In [29]:
# Flatten the output layer to 1 dimension
x_resnet = layers.GlobalMaxPooling2D()(resnet_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_resnet = layers.Dense(512, activation='relu')(x_resnet)
# Add a dropout rate of 0.7
x_resnet = layers.Dropout(0.5)(x_resnet)
# Add a final sigmoid layer for classification
x_resnet = layers.Dense(7, activation='softmax')(x_resnet)

# Configure and compile the model

resnet_model = Model(model_input, x_resnet)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
resnet_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [30]:
resnet_model.load_weights("/content/drive/My Drive/resnet.h5")

In [31]:
resnet_model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 198, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 96, 128, 64)  9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 96, 128, 64)  256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [32]:
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = layers.Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [33]:
ensemble_model = ensemble([resnet_model, inception_model, xception_model], model_input)

In [34]:
ensemble_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [35]:
loss_val, acc_val = ensemble_model.evaluate(X_val, y_val, verbose=1)

26/26 [==============================] - 10s 368ms/step - loss: 0.4431 - accuracy: 0.8753


In [36]:
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

Validation: accuracy = 0.875312  ;  loss_v = 0.443082


In [37]:
loss_test, acc_test = ensemble_model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

63/63 [==============================] - 24s 381ms/step - loss: 0.4871 - accuracy: 0.8592
Test: accuracy = 0.859211  ;  loss = 0.487124
